In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import os
import cv2

In [3]:
input_ex = torch.randn((1,3,244,244))
input_ex.shape

torch.Size([1, 3, 244, 244])

### 데이터 가져오기

In [6]:
from pandas.core.frame import DataFrame
paths = []
dataset_type = []
labels = []

for dirname, _, filenames in os.walk('/content/drive/MyDrive/고모부_머신러닝/dogncat'):
    for filename in filenames:
        filepath = dirname + '/' + filename
        paths.append(filepath)

        if '/training_set' in filepath:
            dataset_type.append('train')

        elif '/test_set' in filepath:
            dataset_type.append('test')

        if 'dogs' in filepath:
            labels.append('DOG')
        
        elif 'cats' in filepath:
            labels.append('CAT')

In [9]:
# Cat n Dog 데이터를 데이터 프레임으로 만들기(path, datatype, label)
cnd_df = pd.DataFrame({'path' : paths, 'type' : dataset_type, 'label' : labels})

# '.jpg'를 포함하는 path만 df로 만들기
filter = cnd_df['path'].str.contains('.jpg') 
cnd_df = cnd_df[filter]

# train, testset으로 분리
train_df = cnd_df[cnd_df['type'] == 'train']
test_df = cnd_df[cnd_df['type'] == 'test']

train_path = train_df['path'].values
test_path = test_df['path'].values

train_label = train_df['label'].values
test_label = test_df['label'].values


In [65]:
onehot_train_label = torch.tensor(pd.get_dummies(train_df['label']).values, dtype=torch.float32)
onehot_test_label = torch.tensor(pd.get_dummies(test_df['label']).values, dtype=torch.float32)


### Custom Dataset만들기

In [20]:
class MyDataset(Dataset):
    def __init__(self, path, label):
        super(MyDataset, self).__init__()
        self.paths = path
        self.labels = label

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        image = cv2.cvtColor(cv2.imread(self.paths[idx]), cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (244,244))
        image = np.asarray(image, dtype=np.float32).transpose(2,0,1)

        if self.labels is not None:
            label = self.labels[idx]

        return image, label



In [67]:
cnd_dataset = MyDataset(train_path, onehot_train_label)
loader = DataLoader(cnd_dataset, batch_size=10, shuffle=True)
i, l = next(iter(loader))
print(i.view(i.size(0), -1))
print(l)

tensor([[ 83.,  54., 137.,  ...,  48.,  69.,  63.],
        [226.,  70.,  70.,  ...,  27.,  28.,  29.],
        [166., 167., 170.,  ...,  12.,  22.,  26.],
        ...,
        [144., 144., 144.,  ...,  75.,  71.,  73.],
        [122., 119., 119.,  ..., 101., 108., 103.],
        [  5.,  10.,  13.,  ...,  12.,  20.,  26.]])
tensor([[0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [0., 1.],
        [1., 0.]])


### 모델 만들기

In [60]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.hidden1 = nn.Conv2d(3, 5, kernel_size=3, padding='same')
        self.relu = nn.ReLU()

        self.fc1 = nn.Linear(3*244*244 ,2)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        print('input shape:', x.shape)
        print(' input max : ', torch.amax(x, dim=(1,2,3)))
        print(' input min : ', torch.amin(x, dim=(1,2,3)))
        print(' input l2norm : ', torch.linalg.vector_norm(x, dim=(1,2,3)))

        # hidden layer
        conv = self.hidden1(x)
        print('conv result shape :', conv.shape)
        print(' conv max : ', torch.amax(conv, dim=(1,2,3)))
        print(' conv min : ', torch.amin(conv, dim=(1,2,3)))
        print(' conv l2norm : ', torch.linalg.vector_norm(conv, dim=(1,2,3)))
        conv_out = self.relu(conv)
        print('conv_out shape :', conv_out.shape)
        print(' conv_out max : ', torch.amax(conv_out, dim=(1,2,3)))
        print(' conv_out min : ', torch.amin(conv_out, dim=(1,2,3)))
        print(' conv_out l2norm : ', torch.linalg.vector_norm(conv_out, dim=(1,2,3)))

        fc_input = x.view(x.size(0), -1)
        print('fc_input shape :', fc_input.shape)
        print(' fc_input max : ', torch.amax(fc_input, dim=1))
        print(' fc_input min : ', torch.amin(fc_input, dim=1))
        print(' fc_input l2norm : ', torch.linalg.vector_norm(fc_input, dim=1))

        # output layer
        fc_logit = self.fc1(fc_input)
        print('fc_logit shape :', fc_logit.shape)
        print(' fc_logit max : ', torch.amax(fc_logit, dim=1))
        print(' fc_logit min : ', torch.amin(fc_logit, dim=1))
        print(' fc_logit l2norm : ', torch.linalg.vector_norm(fc_logit, dim=1))
        fc_output = self.softmax(fc_logit)
        print('fc_output :', fc_output)
        print('fc_output shape :', fc_output.shape)
        print(' fc_output max : ', torch.amax(fc_output, dim=1))
        print(' fc_output min : ', torch.amin(fc_output, dim=1))
        print(' fc_output l2norm : ', torch.linalg.vector_norm(fc_output, dim=1))

        return fc_output
        

In [61]:
model = CNN()
model

CNN(
  (hidden1): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (relu): ReLU()
  (fc1): Linear(in_features=178608, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
)

In [69]:
image_data, label_data = next(iter(loader))
loss_fn = nn.CrossEntropyLoss()
hyphothesis = model(image_data)
loss = loss_fn(hyphothesis, label_data)
print(hyphothesis)
print(loss)

input shape: torch.Size([10, 3, 244, 244])
 input max :  tensor([255., 255., 255., 255., 255., 255., 255., 255., 255., 255.])
 input min :  tensor([ 0.,  0.,  1.,  7., 11.,  6.,  0.,  0.,  0.,  0.])
 input l2norm :  tensor([50322.0391, 62513.4609, 63135.6562, 50102.9766, 55699.6602, 45204.2617,
        48387.0938, 54231.7148, 41467.0664, 33966.2070])
conv result shape : torch.Size([10, 5, 244, 244])
 conv max :  tensor([318.2874, 313.0726, 319.5253, 307.2996, 319.3111, 284.0623, 317.7573,
        319.2509, 318.9292, 317.0482], grad_fn=<AmaxBackward0>)
 conv min :  tensor([ -99.9369, -104.3439,  -84.9921, -116.1526, -104.6851,  -62.9392,
         -77.8961, -102.8595,  -84.3191,  -73.8685], grad_fn=<AminBackward0>)
 conv l2norm :  tensor([49598.9648, 61589.6680, 62359.7188, 49837.2461, 53446.4062, 44562.4219,
        47552.7969, 53735.5352, 40360.1680, 33624.2305],
       grad_fn=<LinalgVectorNormBackward0>)
conv_out shape : torch.Size([10, 5, 244, 244])
 conv_out max :  tensor([318.2874

In [63]:
hyphothesis

tensor([[1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 3.6927e-23],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.0000e+00, 2.4166e-38]], grad_fn=<SoftmaxBackward0>)

In [44]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)

In [47]:
print(input)
print()
print(target)
print(output)

tensor([[-1.2552,  0.0061, -0.0628,  0.1683, -0.8156],
        [ 0.7931, -0.3394,  1.0257,  0.4712,  1.2140],
        [ 0.1355, -1.5505,  0.3422,  0.5980,  0.2279]], requires_grad=True)

tensor([1, 4, 4])
tensor(1.3449, grad_fn=<NllLossBackward0>)


In [48]:
target2 = torch.randn(3,5).softmax(dim=1)
output2 = loss(input, target2)

print(target2)
print(output2)

tensor([[0.2395, 0.1087, 0.0064, 0.1246, 0.5209],
        [0.0881, 0.1126, 0.5270, 0.0960, 0.1763],
        [0.1382, 0.0414, 0.2380, 0.4794, 0.1029]])
tensor(1.6698, grad_fn=<DivBackward1>)


normalized -> 0~1사이의 값으로 만들어주는 것